In [17]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(r'C:\Users\user\innio_case_study\data\northwind.db')
c_df = pd.read_sql_query("select * from customers", conn)
c_df2 = c_df[['CustomerID','City']]
list_cities = c_df2['City'].unique().tolist()
print(list_cities)

['Berlin', 'México D.F.', 'London', 'Luleå', 'Mannheim', 'Strasbourg', 'Madrid', 'Marseille', 'Tsawassen', 'Buenos Aires', 'Bern', 'Sao Paulo', 'Aachen', 'Nantes', 'Graz', 'Lille', 'Bräcke', 'München', 'Torino', 'Lisboa', 'Barcelona', 'Sevilla', 'Campinas', 'Eugene', 'Caracas', 'Rio de Janeiro', 'San Cristóbal', 'Elgin', 'Cork', 'Cowes', 'Brandenburg', 'Versailles', 'Toulouse', 'Vancouver', 'Walla Walla', 'Frankfurt a.M.', 'San Francisco', 'Barquisimeto', 'I. de Margarita', 'Portland', 'Bergamo', 'Bruxelles', 'Montréal', 'Leipzig', 'Anchorage', 'Köln', 'Paris', 'Salzburg', 'Cunewalde', 'Albuquerque', 'Reggio Emilia', 'Genève', 'Stavern', 'Boise', 'Kobenhavn', 'Lander', 'Charleroi', 'Butte', 'Münster', 'Kirkland', 'Århus', None, 'Lyon', 'Reims', 'Stuttgart', 'Oulu', 'Resende', 'Seattle', 'Helsinki', 'Warszawa']


In [ ]:
import requests
import pandas as pd
import requests

geo_data = []

for city in list_cities:
    response = requests.get(
        f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=5&appid=bc2a38adcc749f24fe08205c68bb5c33"
    )
    if response.status_code == 200:
        city_data = response.json()
        if city_data:  # only if non-empty
            for entry in city_data:
                entry['input_city'] = city  # track original city for reference
            geo_data.extend(city_data)
    else:
        print(f"Failed to retrieve data for {city}")

# Create DataFrame after loop

df = pd.DataFrame(geo_data)

In [109]:
# ISO2 to country name mapping
from iso3166 import countries

iso_map = {country.alpha2: country.name for country in countries}
df['country_full'] = df['country'].map(iso_map)

In [ ]:
df = df.rename(columns={'name': 'City'})  # Rename 'name' to 'City' for consistency
# Ensure 'City' is a string type
df['City'] = df['City'].astype(str)     
import unicodedata

def normalize(city):
    if pd.isna(city):
        return None
    # Remove accents, convert to lowercase, strip whitespace
    return ''.join(
        c for c in unicodedata.normalize('NFD', city)
        if unicodedata.category(c) != 'Mn'
    ).lower().strip()

c_df['City'] = c_df['City'].apply(normalize)
df['City'] = df['City'].apply(normalize)

In [116]:
df_unique = df.drop_duplicates(subset=['City', 'country'])

In [ ]:
geo_df = c_df.merge(
    df_unique.set_index(['City', 'country_full']),
    left_on=['City', 'Country'],
    right_index=True,
    how='left'
)

In [ ]:
import requests
import pandas as pd

# Your existing merged customer + geo DataFrame
# Make sure it contains columns: CustomerID, lat, lon
# We'll only use rows that have valid lat/lon
valid_geo_df = geo_df.dropna(subset=['lat', 'lon'])

# Your OpenWeather API key
api_key = "8de9abed422e492a58cf716b0e24caf0"

# Store all API responses
weather_data = []

# Loop through each customer with lat/lon
for idx, row in valid_geo_df.iterrows():
    lat = row['lat']
    lon = row['lon']
    customer_id = row['CustomerID']

    # Build the One Call 3.0 API URL (current + forecast)
    url = f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&appid={api_key}&units=metric"

    try:
        response = requests.get(url)
        if response.status_code == 200:
            result = response.json()
            
            # Track which customer this response belongs to
            result['CustomerID'] = customer_id
            result['lat'] = lat
            result['lon'] = lon

            # Add the API result to our list
            weather_data.append(result)
        else:
            print(f"[ERROR] Failed for CustomerID {customer_id}: Status {response.status_code}")

    except Exception as e:
        print(f"[EXCEPTION] CustomerID {customer_id}: {str(e)}")

# Convert all responses into a DataFrame
# This flattens nested JSON (e.g., current.temp, current.weather[0].description)
weather_df = pd.json_normalize(weather_data)

# Preview the key weather data
print(weather_df[['CustomerID', 'lat', 'lon', 'current.temp', 'current.weather']].head())

# OPTIONAL: Save to CSV or merge with geo_df
# weather_df.to_csv("customer_weather_data.csv", index=False)

# OPTIONAL: Merge weather info back to original customer data
#merged_df = geo_df.merge(weather_df, on=['CustomerID', 'lat', 'lon'], how='left')
#Matbe not rquired, depends on your next steps



In [125]:
print(weather_data)

[]


In [127]:
res1 = f"https://api.openweathermap.org/data/3.0/onecall?lat=52.517037&lon=13.388860&appid=appid=bc2a38adcc749f24fe08205c68bb5c33&units=metric"

In [128]:
res1 = requests.get(res1)
print(res1)

<Response [401]>
